In [1]:
path = '/home/arunava'
#****************************To be Change to reproduce ou result*********************************************

# location of the data and where to store iamge feature image
path_output_chd = path+'/VQA_Med'    

input_vqa_train = 'train_dataset_pickle/train_dataset_df.pkl'
input_vqa_valid ='valid_dataset_pickle/valid_dataset_df.pkl'

img_feat_train = 'train_dataset_pickle/train-image-feature.pickle'
img_feat_valid ='valid_dataset_pickle/valid-image-feature.pickle'

input_test = 'test_dataset_pickle/C1_test_dataset_df.pkl'
img_feat_test = 'test_dataset_pickle/test-image-feature.pickle'

In [2]:
import json
with open(path+'/VQA_Med/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

178


In [3]:
import easydict
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":100,
    })

In [4]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import matplotlib.pyplot as plt
import re
import collections
import pickle
import os
from tqdm import tqdm
import json
import torch.nn as nn
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision
from PIL import Image
import torch.nn.functional as f
import random
from sklearn.preprocessing import LabelEncoder

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [6]:
category_names = {'C1': 'Modality','C2': 'Plane','C3': 'Organ', 'C4': 'Abnormality','All': 'All dataset',}

In [7]:
path_change = path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''

97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [8]:
train_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/'#QAPairsByCategory/'
train_images_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/Train_images/'

valid_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/'#QAPairsByCategory/'
valid_images_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/Val_images/'

test_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/' 
test_images_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/VQAMed2019_Test_Images/'

size = opt.IMG_INPUT_SIZE
size

224

In [9]:
class FeatureExtractor(nn.Module):
    def __init__(self, patch_size = 4):
        super(FeatureExtractor, self).__init__()
        # visual feature extraction
        self.img_feature_extractor = torchvision.models.resnet18(pretrained=True)
        self.img_feature_extractor = torch.nn.Sequential(*(list(self.img_feature_extractor.children())[:-2]))
        self.resize_dim = nn.AdaptiveAvgPool2d((patch_size,patch_size))
        
    def forward(self, img):
        outputs = self.resize_dim(self.img_feature_extractor(img))
        return outputs

In [10]:
qa_path = train_path+'QAPairsByCategory'+'/'+'C1_Modality_train.txt'

In [11]:
columns=['image_id','Question','Answer']

In [12]:
train_df = pd.read_table(qa_path,delimiter='|',header=None)

In [13]:
train_df.columns=columns

In [14]:
classes = train_df['Answer'].unique().tolist()

In [15]:
classes.append('pet-ct fusion')

In [16]:
classes_dict={k: v for v, k in enumerate(classes)}

In [17]:
classes_dict

{'cta - ct angiography': 0,
 'no': 1,
 'us - ultrasound': 2,
 'xr - plain film': 3,
 'noncontrast': 4,
 'yes': 5,
 't2': 6,
 'ct w/contrast (iv)': 7,
 'mr - flair': 8,
 'mammograph': 9,
 'ct with iv contrast': 10,
 'gi and iv': 11,
 't1': 12,
 'mr - t2 weighted': 13,
 'mr - t1w w/gadolinium': 14,
 'contrast': 15,
 'iv': 16,
 'an - angiogram': 17,
 'mra - mr angiography/venography': 18,
 'nm - nuclear medicine': 19,
 'mr - dwi diffusion weighted': 20,
 'ct - gi & iv contrast': 21,
 'ct noncontrast': 22,
 'mr - other pulse seq.': 23,
 'ct with gi and iv contrast': 24,
 'flair': 25,
 'mr - t1w w/gd (fat suppressed)': 26,
 'ugi - upper gi': 27,
 'mr - adc map (app diff coeff)': 28,
 'bas - barium swallow': 29,
 'pet - positron emission': 30,
 'mr - pdw proton density': 31,
 'mr - t1w - noncontrast': 32,
 'be - barium enema': 33,
 'us-d - doppler ultrasound': 34,
 'mr - stir': 35,
 'mr - flair w/gd': 36,
 'ct with gi contrast': 37,
 'venogram': 38,
 'mr t2* gradient,gre,mpgr,swan,swi': 39,


In [18]:
with open('c1_answer_classes.json', 'w') as fp:
        json.dump(classes_dict, fp)

In [19]:
print(len(classes))

45


In [20]:
train_df['labels'] = train_df['Answer'].apply(lambda x: classes_dict[x] )
train_df['image_path'] = train_df['image_id'].apply(lambda x: f"{train_images_path}{x}.jpg" )

In [21]:
train_df['image_path'].head(5)

0    /home/arunava/VQA_Med_2019_Dataset/Train/Image...
1    /home/arunava/VQA_Med_2019_Dataset/Train/Image...
2    /home/arunava/VQA_Med_2019_Dataset/Train/Image...
3    /home/arunava/VQA_Med_2019_Dataset/Train/Image...
4    /home/arunava/VQA_Med_2019_Dataset/Train/Image...
Name: image_path, dtype: object

In [22]:
transform = {
    phase: transforms.Compose([transforms.Resize((300,256)), 
                                # transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))]) 

    for phase in ['train', 'valid']}

test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])
train_transform = transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

valid_transform = transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])


In [23]:
class FeatureExtractor(nn.Module):
    def __init__(self, patch_size = 4):
        super(FeatureExtractor, self).__init__()
        # visual feature extraction
        self.img_feature_extractor = torchvision.models.resnet18(pretrained=True)
        self.img_feature_extractor = torch.nn.Sequential(*(list(self.img_feature_extractor.children())[:-2]))
        self.resize_dim = nn.AdaptiveAvgPool2d((patch_size,patch_size))
        
    def forward(self, img):
        outputs = self.resize_dim(self.img_feature_extractor(img))
        return outputs


In [24]:
feature_network = FeatureExtractor(patch_size = 5).to(device)

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
feature_network.eval()

FeatureExtractor(
  (img_feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [26]:
class VqaDataset(data.Dataset):
    '''
        Main class use to retrieve our dataset from pickle file.
    '''

    def __init__(self, input_vqa, img_feat_vqa, transform=None, phase = 'train'):
        # self.input_dir = input_dir
        # self.vqa = np.load(input_dir+'/'+input_vqa, allow_pickle=True )
        # self.img_feat_vqa = np.load(input_dir+'/'+img_feat_vqa, allow_pickle=True )
        self.vqa = input_vqa
        self.vocab_size = None
        self.phase = phase
        self.transform = transform
   

    def __getitem__(self, idx):

        vqa = self.vqa
        phase = self.phase
        transform=self.transform[phase]
        # img_feat_vqa = self.img_feat_vqa
        
        image_id = vqa['image_id'].values[idx]
        image_path = vqa['image_path'].values[idx]
        # image_feat = torch.Tensor(img_feat_vqa[image_id])
        # print('image_path',image_path)
        
        image = Image.open(image_path)
        # print('transform',transform)
        if transform:
            image = transform(image)

        image = torch.unsqueeze(image, 0).to(device)
        qst2idc =  vqa['Question'].values[idx]
        sample = { 'question': qst2idc, 'image': image, 'image_id':image_id, 'image_path': image_path } 
        if (self.phase == 'train') or  (self.phase == 'valid'):
            ans2idc = vqa['labels'].values[idx]
            answer_text = vqa['Answer'].values[idx]
            sample['image_id'] = image_id
            sample['label'] = ans2idc
            sample['answer_text'] = answer_text
        else:
            sample['image_id'] = image_id
            
            
        return sample

    def __len__(self):

        return len(self.vqa)


def get_loader(input_vqa_train, input_vqa_valid, batch_size, num_workers, transform=None,size=228):
    '''
        Load our dataset with dataloader for the train and valid data
    '''

    vqa_dataset = {
        'train': VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_vqa_train,
            img_feat_vqa=img_feat_train,
            transform = transform,
            phase = 'train'),
        'valid': VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_vqa_valid,
            img_feat_vqa=img_feat_valid,
            transform = transform,
            phase = 'valid')}
    

    data_loader = {
        phase: torch.utils.data.DataLoader(
            dataset=vqa_dataset[phase],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            )
        for phase in ['train','valid']}

    return data_loader


def get_test_loader(input_test, batch_size, num_workers,size=228):

   
    
    test_vqa_dataset = VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_test,
            # img_feat_vqa=img_feat_vqa,
            phase = 'test')
    
    
    data_loader = torch.utils.data.DataLoader(dataset=test_vqa_dataset,
                                                transform=transform,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                num_workers=num_workers)
    return data_loader

In [27]:
input_vqa_train = train_df
# input_vqa_valid = valid_dataset_df

saved_dir = path_output_chd

num_epochs = opt.NUM_EPOCHS
image_size = opt.IMG_INPUT_SIZE
num_workers = 0
batch_size = opt.BATCH_SIZE

In [28]:
data_loader = get_loader(
        input_vqa_train = train_df, 
        input_vqa_valid = input_vqa_valid,
        batch_size = batch_size, 
        num_workers = num_workers,
        transform = transform,
        size = image_size )

In [29]:
def mixup_data_vqa(id,v,p, q, a, al, alpha=1.0, use_cuda=True):
    '''Returns mixed images, pairs of questions, targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = v.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).to(device)
    else:
        index = torch.randperm(batch_size)
    # print('index',index)
    mixed_v = lam * v + (1 - lam) * v[index, :]
    # print('mixed_v',mixed_v.shape)
    id_a, id_b = id,id[index]
    p_a,p_b = p,p[index]
    a_a, a_b = a, a[index]
    al_a,al_b = al,al[index]
    # print('a_a',a_a)
    # print('a_b',a_b)
    q_a, q_b = q, q[index]
    # print('q_a',q_a)
    # print('q_b',q_b)
    
    return id_a,id_b,mixed_v,p_a,p_b, a_a, a_b,al_a,al_b, q_a, q_b, lam

In [30]:
image_feat = {}
dict_data={'image_id': [],  'image_path_a': [], 'image_path_b':[] ,'question_a': [], 'question_b':[], 'labels_a':[] , 'labels_b':[], 'answer_a' : [], 'answer_b':[], 'lamda':[]  }

In [31]:
for batch_idx, batch_sample in enumerate(data_loader['train']):
    image_id = batch_sample['image_id']
    image_path = batch_sample['image_path']
    image = batch_sample['image'].to(device)
    questions = batch_sample['question']
    labels = batch_sample['label'].to(device)
    labelss = batch_sample['label']
    labelss = labelss.cpu().data.numpy()
    label_answer_text = batch_sample['answer_text']
    le1 = LabelEncoder()
    label_answers_text_l = le1.fit_transform(label_answer_text)
    label_answers_text_l = torch.as_tensor(label_answers_text_l).to(device)
    le2 = LabelEncoder() 
    question_labels = le2.fit_transform(questions)
    question_labels = torch.as_tensor(question_labels).to(device)
    le3 = LabelEncoder()
    image_id_labels = le3.fit_transform(image_id)
    image_id_labels = torch.as_tensor(image_id_labels).to(device)
    le4 = LabelEncoder()
    image_path_labels = le4.fit_transform(image_path)
    image_path_labels = torch.as_tensor(image_path_labels).to(device)

    for index,img in enumerate(image):
        with torch.no_grad():
            visual_features = feature_network(img)
            visual_features = torch.flatten(visual_features, start_dim=2)
            visual_features = visual_features.permute((0,2,1))
            visual_features = visual_features.squeeze(0)
            visual_features = visual_features.cpu().numpy()
            image_feat[image_id[index]]=visual_features
        
        dict_data['image_id'].append(image_id[index])
        dict_data['image_path_a'].append(image_path[index])
        dict_data['image_path_b'].append(image_path[index])
        dict_data['question_a'].append(questions[index])
        dict_data['question_b'].append(questions[index])
        dict_data['answer_a'].append(label_answer_text[index])
        dict_data['answer_b'].append(label_answer_text[index])
        dict_data['labels_a'].append(labelss[index])
        dict_data['labels_b'].append(labelss[index])
        dict_data['lamda'].append(1.0)

    id_a,id_b,mixed_v, p_a,p_b,a_a, a_b,al_a,al_b, q_a, q_b, lam = mixup_data_vqa(
                image_id_labels,image, image_path_labels,question_labels, labels, label_answers_text_l, alpha=0.1, use_cuda=True)

    # mixed_v = mixed_v.cpu().data.numpy()
    a_a = a_a.cpu().data.numpy()
    a_b = a_b.cpu().data.numpy()
    id_a = id_a.cpu().data.numpy()
    id_b = id_b.cpu().data.numpy()
    q_a = q_a.cpu().data.numpy()
    q_b = q_b.cpu().data.numpy()
    al_a = al_a.cpu().data.numpy() 
    al_b = al_b.cpu().data.numpy()
    p_a = p_a.cpu().data.numpy()
    p_b = p_b.cpu().data.numpy()
    p_a_l = le4.inverse_transform(p_a)
    p_b_l = le4.inverse_transform(p_b)
    id_a_l = le3.inverse_transform(id_a)
    id_b_l = le3.inverse_transform(id_b)
    q_a_l = le2.inverse_transform(q_a)
    q_b_l = le2.inverse_transform(q_b)
    al_a_l = le1.inverse_transform(al_a)
    al_b_l = le1.inverse_transform(al_b)
    
    

    for index,img in enumerate(mixed_v):
        # print('img_shape',img.shape)
        # mix_image_feature = vgg19_model(img[None,...]).to(device)
        with torch.no_grad():
            mix_visual_features = feature_network(img)
            mix_visual_features = torch.flatten(mix_visual_features, start_dim=2)
            mix_visual_features = mix_visual_features.permute((0,2,1))
            mix_visual_features = mix_visual_features.squeeze(0)
            # visual_features = mix_visual_features.data.cpu().numpy()
            # print('mix_img_feature.shape',mix_image_feature.shape)
            image_feat[id_a_l[index]+'_'+id_b_l[index]] = mix_visual_features.cpu().numpy()
        # image_feat_train[id_a_l[index]+'_'+id_b_l[index]] = mix_visual_features.data.cpu().numpy()

    # imshow(torchvision.utils.make_grid(mixed_v))
    mixed_v = mixed_v.cpu().detach().numpy()
    
    for index in range(len(mixed_v)):
        # print(id_a[index])
        image_name = id_a_l[index]+'_'+id_b_l[index]
        dict_data['image_id'].append(image_name)
        # dict_data['image_id'].append(id_a_l[index]+'_'+id_b_l[index])
        # dict_data['mixed_image'].append(mixed_v[index])
        # dict_data['mixed_image'].append(mixed_v[index])
        dict_data['image_path_a'].append(p_a_l[index])
        dict_data['image_path_b'].append(p_b_l[index])
        dict_data['question_a'].append(q_a_l[index])
        dict_data['question_b'].append(q_b_l[index])
        dict_data['labels_a'].append(a_a[index])
        dict_data['labels_b'].append(a_b[index])
        dict_data['answer_a'].append(al_a_l[index])
        dict_data['answer_b'].append(al_b_l[index])
        
        dict_data['lamda'].append(lam)
        # mixed_image = np.transpose(mixed_v[index], (1, 2, 0))
        # print(mixed_image.min())
        # mini = mixed_image.min()
        # maxi = mixed_image.max()
        # print(mixed_image.max())
        # print(mini)
        # print(maxi)
        # plt.imshow(mixed_image)
        # new_mixed_image = (1/(mini*(-1)*maxi)) * mixed_image + 0.5
        # new_mixed_image = mixed_image/(maxi-mini) + 0.5
        # plt.imsave('myy.png',(mixed_image).astype(np.uint8),cmap='gray')
        # fig, axs = plt.subplots(1, 3, figsize=(9, 3))
        # axs[0].imshow(Image.open(p_a_l[index]).convert('RGB'))
        # axs[0].set_title(id_a_l[index])
        # axs[1].imshow(Image.open(p_b_l[index]).convert('RGB'))
        # axs[1].set_title(id_b_l[index])
        # axs[2].imshow(mixed_image)
        # axs[2].set_title(image_name)
        # plt.show()
        # fig.savefig(path+'/VQA_Med_2019_Dataset/Train/mixed/'+image_name+'.png',bbox_inches='tight',pad_inches=1)

        # plt.close()



In [32]:
train_dict = train_df.to_dict('list')
print('train_dict',train_dict.keys())

train_dict dict_keys(['image_id', 'Question', 'Answer', 'labels', 'image_path'])


In [33]:
print(dict_data['image_id'])
print(dict_data['image_path_a'])
print(dict_data['image_path_b'])
print(dict_data['question_a'])
print(dict_data['question_b'])
print(dict_data['answer_a'])
print(dict_data['answer_b'])




['synpic33990', 'synpic27913', 'synpic52990', 'synpic49802', 'synpic27745', 'synpic29147', 'synpic13033', 'synpic27244', 'synpic47420', 'synpic34193', 'synpic60725', 'synpic29733', 'synpic28492', 'synpic37607', 'synpic17883', 'synpic38482', 'synpic53956', 'synpic35143', 'synpic39293', 'synpic37401', 'synpic52906', 'synpic32612', 'synpic51773', 'synpic29781', 'synpic32364', 'synpic25203', 'synpic33273', 'synpic20863', 'synpic33494', 'synpic60661', 'synpic41562', 'synpic36880', 'synpic35231', 'synpic41680', 'synpic48215', 'synpic20710', 'synpic45988', 'synpic14404', 'synpic39849', 'synpic41335', 'synpic29220', 'synpic58377', 'synpic55430', 'synpic55773', 'synpic2029', 'synpic36390', 'synpic34447', 'synpic54547', 'synpic38930', 'synpic45952', 'synpic57196', 'synpic35807', 'synpic53083', 'synpic50678', 'synpic33882', 'synpic21158', 'synpic39634', 'synpic51600', 'synpic24928', 'synpic18561', 'synpic55522', 'synpic52370', 'synpic59120', 'synpic41697', 'synpic33990_synpic41562', 'synpic27913_

In [34]:
print(dict_data['labels_a'])


[array(1), array(12), array(16), array(1), array(5), array(1), array(16), array(5), array(1), array(2), array(25), array(2), array(1), array(8), array(5), array(3), array(5), array(3), array(15), array(27), array(5), array(15), array(3), array(3), array(1), array(4), array(5), array(4), array(3), array(4), array(5), array(16), array(17), array(1), array(3), array(5), array(7), array(3), array(3), array(5), array(5), array(3), array(0), array(5), array(9), array(15), array(3), array(3), array(2), array(5), array(8), array(13), array(5), array(5), array(25), array(4), array(1), array(23), array(19), array(16), array(15), array(12), array(29), array(1), 1, 12, 16, 1, 5, 1, 16, 5, 1, 2, 25, 2, 1, 8, 5, 3, 5, 3, 15, 27, 5, 15, 3, 3, 1, 4, 5, 4, 3, 4, 5, 16, 17, 1, 3, 5, 7, 3, 3, 5, 5, 3, 0, 5, 9, 15, 3, 3, 2, 5, 8, 13, 5, 5, 25, 4, 1, 23, 19, 16, 15, 12, 29, 1, array(8), array(1), array(12), array(3), array(3), array(3), array(18), array(3), array(1), array(1), array(5), array(1), array(3),

In [35]:
mixed_train_df_data = pd.DataFrame(dict_data, columns = ['image_id',
                                                         'image_path_a',
                                                         'image_path_b',
                                                    'question_a',
                                                    'question_b',
                                                    'answer_a',
                                                    'answer_b',
                                                    'labels_a',
                                                    'labels_b',
                                                    'lamda'
                                                ])

In [36]:
mixed_train_df_data

image_id  \
0                 synpic33990   
1                 synpic27913   
2                 synpic52990   
3                 synpic49802   
4                 synpic27745   
...                       ...   
6395  synpic60523_synpic25649   
6396  synpic21598_synpic60523   
6397  synpic19353_synpic17223   
6398  synpic35908_synpic44105   
6399  synpic17223_synpic49274   

                                           image_path_a  \
0     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
1     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
2     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
3     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
4     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
...                                                 ...   
6395  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6396  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6397  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6398  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6399  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   

                                           image_path_b  \
0     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
1     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
2     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
3     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
4     /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
...                                                 ...   
6395  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6396  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6397  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6398  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
6399  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   

                                             question_a  \
0                          is this a t1 weighted image?   
1               what is the mr weighting in this image?   
2          what type of contrast did this patient have?   
3                                    is this a ct scan?   
4                 was iv contrast given to the patient?   
...                                                 ...   
6395            what type of imaging modality is shown?   
6396                       is this a t1 weighted image?   
6397                      what imaging method was used?   
6398  what imaging modality was used to take this im...   
6399              in what modality is this image taken?   

                                             question_b  \
0                          is this a t1 weighted image?   
1               what is the mr weighting in this image?   
2          what type of contrast did this patient have?   
3                                    is this a ct scan?   
4                 was iv contrast given to the patient?   
...                                                 ...   
6395            with what modality is this image taken?   
6396            what type of imaging modality is shown?   
6397              in what modality is this image taken?   
6398  what type of imaging modality is used to acqui...   
6399                              is this an mri image?   

                        answer_a              answer_b labels_a labels_b  \
0                             no                    no        1        1   
1                             t1                    t1       12       12   
2                             iv                    iv       16       16   
3                             no                    no        1        1   
4                            yes                   yes        5        5   
...                          ...                   ...      ...      ...   
6395          ct w/contrast (iv)       xr - plain film        7        3   
6396                          no    ct w/contrast (iv)        1        7   
6397                  mammograph            mammograph        9        9   
6398  ct with gi and iv cont

In [37]:
mixed_train_df_data.shape

(6400, 10)

In [38]:
pickle_name='c1_mixed_train_dataset_df'
mixed_train_df_data.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')

In [39]:
with open(path_output_chd+'/train_dataset_pickle/c1-mixed-train-image-feature.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)

In [40]:
valid_qa_path = valid_path+'QAPairsByCategory'+'/'+'C1_Modality_val.txt'

In [41]:
valid_df = pd.read_table(valid_qa_path,delimiter='|',header=None)

In [42]:
columns=['image_id','Question','Answer']

In [43]:
valid_df.columns=columns

In [44]:
valid_df.shape

(500, 3)

In [45]:
valid_df.head(5)

image_id                                           Question  \
0  synpic54733  what imaging modality was used to take this im...   
1  synpic25647                         what kind of scan is this?   
2  synpic35681                   was the mri taken with contrast?   
3  synpic39641  is this a t1 weighted, t2 weighted, or flair i...   
4  synpic35693                         is this a noncontrast mri?   

                Answer  
0  ct with iv contrast  
1      xr - plain film  
2                   no  
3                   t2  
4                  yes

In [46]:
valid_df['labels'] = valid_df['Answer'].apply(lambda x: classes_dict[x] )
valid_df['image_path'] = valid_df['image_id'].apply(lambda x: f"{valid_images_path}{x}.jpg" )

In [47]:
valid_df.shape

(500, 5)

In [48]:
pickle_name='c1_val_dataset_df'
valid_df.to_pickle(path_output_chd+'/valid_dataset_pickle/'+pickle_name+'.pkl')

In [49]:
valid_image_feat={}

In [50]:
for row in valid_df.itertuples():
    # print(row)
    img = Image.open(row.image_path)
    img = transform['valid'](img)
    img = torch.unsqueeze(img, 0).to(device)
    with torch.no_grad():
        visual_features = feature_network(img)
        visual_features = torch.flatten(visual_features, start_dim=2)
        visual_features = visual_features.permute((0,2,1))
        visual_features = visual_features.squeeze(0)
        visual_features = visual_features.data.cpu().numpy()
        valid_image_feat[row.image_id]=visual_features

In [51]:
with open(path_output_chd+'/valid_dataset_pickle/c1-valid-image-feature.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(valid_image_feat, f, pickle.HIGHEST_PROTOCOL)